First of all, make sure your environment has the latest version of [🤗 Optimum Graphcore](https://github.com/huggingface/optimum-graphcore) installed.

In [ ]:
%pip install git+https://github.com/huggingface/optimum-graphcore.git

Also make sure all the packages required for this notebook are installed.

In [ ]:
%pip install scikit-learn;
%pip install datasets
%pip install evaluate
%pip install tokenizers
%pip install matplotlib
%pip install scipy
%pip install --force-reinstall huggingface_hub==0.11.1;

Let's start by importing and printing out the versions of `Transformers` and `Optimum Graphcore`:

In [ ]:
import transformers
import optimum.graphcore

print(transformers.__version__)
print(optimum.graphcore.__version__)

At the end of this notebook, to be able to share your model with the community and easily access it through HuggingFace, there are some short set-up steps you must follow to enable uploading your checkpoint to the HuggingFace Hub.

First you have to store your authentication token from the Hugging Face website ([sign up here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Git-lfs must also be installed to enable large file storage when pushing to the hub:

In [ ]:
! apt install git-lfs

# Faster single-label text classification with PackedBERT 

This notebook builds on the process of [fine-tuning BERT on a text classification task](text_classification.ipynb), using [packing](https://www.graphcore.ai/posts/introducing-packed-bert-for-2x-faster-training-in-natural-language-processing), an optimisation method originally used for 2x faster BERT pre-training, which can now also provide massive throughput increases for fine-tuning and batched inference! 

**So, what *is* packing?** The basic idea of 'packing' a dataset is to utilise the requirement for constant-shaped inputs into a model. Instead of padding it with empty, unused space, we can recycle this unused space and fill it with more inputs! The architecture of transformer models like BERT supports this, and lets us optimally use this space to process multiple sequences within one input.

**And here is why you might want to use it:** Having a single input contain multiple sequences leads to multiple sequences being processed in parallel in a single pass within a single iteration inside a batch, increasing the 'effective' batch size of the model by a considerable factor in many cases, and most importantly, increasing model throughput for training and batched inference significantly.

This notebook outlines how to easily enable packing for BERT when performing fine-tuning/inference on a text-classification task in 🤗 Optimum, resulting in an impressive 5-6x faster training and inference run-time on the `GLUE/sst2` dataset. 

You can read more about packing in the original [paper](https://arxiv.org/abs/2107.02027).

In this notebook, we will see how to fine-tune BERT, a [🤗 Transformers](https://github.com/huggingface/transformers) model to a text classification task of the [GLUE Benchmark](https://gluebenchmark.com/).

The GLUE Benchmark is a group of nine classification tasks on sentences or pairs of sentences, which are:

- [CoLA](https://nyu-mll.github.io/CoLA/) (Corpus of Linguistic Acceptability) Determine if a sentence is grammatically correct or not.is a  dataset containing sentences labeled grammatically correct or not.
- [MNLI](https://arxiv.org/abs/1704.05426) (Multi-Genre Natural Language Inference) Determine if a sentence entails, contradicts or is unrelated to a given hypothesis. (This dataset has two versions, one with the validation and test set coming from the same distribution, another called mismatched where the validation and test use out-of-domain data.)
- [MRPC](https://www.microsoft.com/en-us/download/details.aspx?id=52398) (Microsoft Research Paraphrase Corpus) Determine if two sentences are paraphrases from one another or not.
- [QNLI](https://rajpurkar.github.io/SQuAD-explorer/) (Question-answering Natural Language Inference) Determine if the answer to a question is in the second sentence or not. (This dataset is built from the SQuAD dataset.)
- [QQP](https://data.quora.com/First-Quora-Dataset-Release-Question-Pairs) (Quora Question Pairs2) Determine if two questions are semantically equivalent or not.
- [RTE](https://aclweb.org/aclwiki/Recognizing_Textual_Entailment) (Recognizing Textual Entailment) Determine if a sentence entails a given hypothesis or not.
- [SST-2](https://nlp.stanford.edu/sentiment/index.html) (Stanford Sentiment Treebank) Determine if the sentence has a positive or negative sentiment.
- [STS-B](http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark) (Semantic Textual Similarity Benchmark) Determine the similarity of two sentences with a score from 1 to 5.
- [WNLI](https://cs.nyu.edu/faculty/davise/papers/WinogradSchemas/WS.html) (Winograd Natural Language Inference) Determine if a sentence with an anonymous pronoun and a sentence with this pronoun replaced are entailed or not. (This dataset is built from the Winograd Schema Challenge dataset.)

We will see how to easily load the dataset for these tasks and use BERT with packing to fine-tune a model on SST-2. Each task is named using an acronym, with `mnli-mm` standing for the 'mis-matched' version of MNLI (so it is the same training set as `mnli` but with different validation and test sets):

In [ ]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

**For this Packed BERT demo, we cover (single-label) sequence classification on the `sst2` dataset. The `task` can be changed to run the other `GLUE` tasks. However, training hyperparameters may need further tuning for these other tasks.**

Let's initialise our training configurations. 

In this notebook, we are using both data parallelism and pipeline parallelism (see this [tutorial](https://github.com/graphcore/tutorials/tree/master/tutorials/pytorch/tut2_efficient_data_loading) for more). Therefore the global batch size, which is the actual number of samples used for the weight update, is determined using four factors:

    global batch size = micro_batch_size * gradient accumulation steps * device iterations * replication factor

Replication factor is determined by `pod_type`, which will be used as a key to select the replication factor from a dictionary defined in the IPU config file. For example, the dictionary in the IPU config file [Graphcore/roberta-base-ipu](https://huggingface.co/Graphcore/roberta-base-ipu/blob/main/ipu_config.json) looks like this.:

    "replication_factor": {"pod4": 1, "pod8": 2, "pod16": 4, "pod32": 8, "pod64": 16, "default": 1}

Depending on your model and the pod machine you are using, you might need to adjust these three batch-size-related arguments.

By default this notebook is configured to run on 4 IPUs.

Finally, `max_seq_length` is the maximum length a sequence can be, and all sequences will be padded to this length, so it should not be larger than the maximum length of the model. Set these parameters and the rest of the notebook should run smoothly:

Given the small size of the sequences in `sst2`, we can reduce the model maximum input size to `max_seq_length = 256`.

In [ ]:
task = "sst2"
model_checkpoint = "bert-base-uncased"
ipu_config_name = "Graphcore/bert-base-uncased"
micro_batch_size = 2
gradient_accumulation_steps = 32
device_iterations = 32
max_seq_length = 256

Gradients are not calculated during validation, so gradient accumulation is not applicable, and the global batch size for validation can be defined separately as:

```
global_validation_batch_size=device_iterations*replication_factor*max_seq_per_pack
```



Values for machine size and cache directories can be configured through environment variables or directly in the notebook:

In [ ]:
import os

pod_type = os.getenv("GRAPHCORE_POD_TYPE", "pod4")
executable_cache_dir = os.getenv("POPLAR_EXECUTABLE_CACHE_DIR", "./exe_cache/") + "/packed_bert_slseqcls/"

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library and the  [🤗 Evaluate](https://github.com/huggingface/evaluate) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `evaluate.load()`.  

In [ ]:
from datasets import load_dataset
import evaluate

Apart from `mnli-mm` being a special code, we can directly pass our task name to those functions. `load_dataset` will cache the dataset to avoid downloading it again the next time you run this cell.

In [ ]:
actual_task = "mnli" if task == "mnli-mm" else task
dataset = load_dataset("glue", actual_task)
metric = evaluate.load('glue', actual_task)

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set (with more keys for the mismatched validation and test set in the special case of `mnli`).

In [ ]:
dataset

To access an actual element, you need to select a split first, then give an index:

In [ ]:
dataset["train"][0]

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset["train"])

The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [ ]:
metric

You can call its `compute` method with your predictions and labels directly and it will return a dictionary with the metric(s) value:

In [ ]:
import numpy as np

fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)

Note that `load_metric` has loaded the proper metric associated to your task, which is:

- for CoLA: [Matthews Correlation Coefficient](https://en.wikipedia.org/wiki/Matthews_correlation_coefficient)
- for MNLI (matched or mismatched): Accuracy
- for MRPC: Accuracy and [F1 score](https://en.wikipedia.org/wiki/F1_score)
- for QNLI: Accuracy
- for QQP: Accuracy and [F1 score](https://en.wikipedia.org/wiki/F1_score)
- for RTE: Accuracy
- for SST-2: Accuracy
- for STS-B: [Pearson Correlation Coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) and [Spearman's_Rank_Correlation_Coefficient](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient)
- for WNLI: Accuracy

so the metric object only computes the one(s) needed for your task.

## Preprocessing the data

Before we can feed the texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [ ]:
tokenizer("Hello, this is one sentence!", "And this sentence goes with it.")

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

To preprocess our dataset, we will thus need the names of the columns containing the sentence(s). The following dictionary keeps track of the correspondence task to column names:

In [ ]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

We can double check it does work on our current dataset:

In [ ]:
sentence1_key, sentence2_key = task_to_keys[task]

if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the three arguments.`truncation=True` will ensure that an input longer than maximum length will be truncated to the maximum length. `max_length=max_seq_length` sets the maximum length of a sequence.

**Important: since we will use packing later, we don't want to perform any padding in the tokenizer.**

In [ ]:
# no padding for packing
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True, max_length=max_seq_length)
    
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True, max_length=max_seq_length)

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
preprocess_function(dataset['train'][:5])

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True)
len(encoded_dataset['train'])

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Packing the dataset

To implement packing, we need to pack our dataset first. Each new element will be a "pack" containing at most `max_seq_per_pack` sequences.

In [ ]:
max_seq_per_pack = 6

We also define the number of labels in our dataset, `sst2` is a single_label task: it will contain one true class for each example.

In [ ]:
num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2
problem_type = 'single_label_classification'

### Packing algorithm

In order to pack efficiently, we will use a histogram-based algorithm: shortest-pack-first histogram packing (SPFHP) presented in the [blog post](https://www.graphcore.ai/posts/introducing-packed-bert-for-2x-faster-training-in-natural-language-processing) adapted from the [blog code](https://github.com/graphcore/tutorials/tree/master/blogs_code/packedBERT). The full process of packing the dataset consists of four steps:

1. Create a histogram of the sequence lengths of the dataset.
2. Generate the 'strategy' for the dataset using one of the state-of-the-art packing algorithms, which maps out the order and indices of the sequences that need to be packed together.
3. Use this strategy to create the actual dataset, concatenating the tokenized features together for each column in the dataset, including the labels.
4. Finally, pass these new columns into a custom PyTorch dataset, ready to be passed to the PopTorch dataloader!

These steps have been simplified through the easy-to-use `utils.packing` available in Graphcore Optimum. You can simply generate the packed dataset after the usual tokenization and preprocessing by passing all necessary packing configuration to the `PackedDatasetCreator` class, and generate the ready-to-use PyTorch dataset with `.create()`.

Within the function, there are some column names used by default. The expected default columns for text classification include:
* `input_ids`
* `attention_mask`
* `token_type_ids`
* `labels`

These should all be generated automatically when tokenizing any classification dataset for BERT. However, the labels key, as it is not encoded, may have a different name. For this dataset, the column key for the labels for this dataset is `label`, since the dataset creator expects `labels`, we can pass this to the argument `custom_label_key`, so the class can find our labels. 

The `PackedDatasetCreator` requires different instantiations for different datasets, so it must be called separately for each of our dataset splits. We can set either `training`, `validation` or `inference` to `True` as needed.

In [ ]:
from utils.packing.dataset_creator import PackedDatasetCreator

train_data_packer = PackedDatasetCreator(
    tokenized_dataset = encoded_dataset['train'],
    max_sequence_length = max_seq_length,
    max_sequences_per_pack = max_seq_per_pack,
    training = True,
    num_labels = num_labels,
    problem_type = problem_type,
    algorithm = 'SPFHP',
    custom_label_key = 'label'
)

val_data_packer = PackedDatasetCreator(
    tokenized_dataset = encoded_dataset['validation'],
    max_sequence_length = max_seq_length,
    max_sequences_per_pack = max_seq_per_pack,
    validation = True,
    num_labels = num_labels,
    problem_type = problem_type,
    algorithm = 'SPFHP',
    custom_label_key = 'label'
)

This will create the strategy and initialise the necessary parameters for packing the dataset. We can see that the ideal speed-up we have achieved is approximately 5.15x the original dataset, which corresponds directly to the average packing factor: the average number of sequences within one pack.

The `PackedDatasetCreator` class also has some other features we do not use here for training, such as `pad_to_global_batch_size`, a feature useful for performing batched inference on a large samples when we do not want to lose any of the samples when creating data iterators using the `poptorch.Dataloader`, it applies 'vertical' padding to the dataset, adding filler rows to bring the dataset up to a value divisible by the global batch size, and allows for the largest possible batch sizes to be used without any loss of data.

You can also view the histogram generated in the first step of the packing process, to observe whether the distribution of sequence lengths in the dataset will benefit from packing - as a general rule, as long as the average length of the sequences in the dataset is 50% or less of the maximum sequence length, packing will offer at least a 2x throughput benefit, in other words: `throughput_increase ≈ max_seq_len/mean_seq_len`

Many datasets have distributions with much smaller average lengths, and will benefit much more. We can easily observe this distribution by retrieving and plotting the histogram from the data class:

In [ ]:
import matplotlib.pyplot as plt

train_histogram = train_data_packer.histogram

plt.hist(train_histogram, bins = [k for k in range(0,max_seq_length,10)]) 
plt.title("Sequence length histogram") 
plt.xlabel('Sequence lengths')
plt.ylabel('Frequency')
plt.show()

Now we need to create the actual packed dataset, this is the 3rd step of the packing process outlined above.

In this stage, we take the strategy for mapping the sequences by size into 'packs' that was generated by the packing algorithm, and use this to extract the sequences from the tokenized dataset, inserting them into packs for each column in the dataset. Any remaining space in a pack after the sequences have been concatenated is padded to bring all sequences up to the maximum sequence length.

Some key features unique to packed datasets are worth mentioning here:

- A specific `attention_mask` is generated: It contains a unique index for each sequence of the pack and `0` for the remaining padding tokens. This, essentially, tells the model where to "look" from the perspective of a single token, ignoring any encoded information (such as a different sequence) that is not relevant to that token.
    - Example of 3 sequences: `attention_mask = [1,1,1,1,1,1,2,2,2,2,2,3,3,3,3,3,0,...,0,1,2,3]`


- The [CLS] tokens of each sequence must be moved to the end of the pack.
    - For instance: `[CLS,a,b,c] + [CLS, d,e,f] + [CLS, g,h,i] -> [a,b,c,d,e,f,g,h,i,...,CLS,CLS,CLS]`
    

- The `position_ids` of a pack contain the concatenated `position_ids` of each sequences 
    - For instance given 3 sequences: `[0,1,2,3,4] + [0,1,2,3] + [0,1,2] -> [1,2,3,4,1,2,3,1,2,...,0,0,0]` (note: the CLS tokens position id '0' are also moved the end of the pack)
    
- `labels` and `token_type_ids` are also packed to correspond to the `input_ids` pack.


To create a dataloader-ready packed dataset, all you need to do is call the `create()` method:

In [ ]:
packed_train_dataset = train_data_packer.create()
packed_val_dataset = val_data_packer.create()

Let's visualize one sample of the new `packed_train_dataset`:

In [ ]:
packed_train_dataset[133]

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it.

### Implement Packed BERT

A few model modifications are required to make packing work with BERT.
We extend the existing class `BertForSequenceClassification` to `PipelinedPackedBertForSequenceClassification` which incorporates the required changes to the pooler and the model output. The crux of these changes is to modify the generic sequence classification model to handle 'unpacking' multiple sequences in the output stage, treating them as a larger batch size for classification, as well as masking any padding created by packing.

First let's load a default BERT configuration using `AutoConfig`. The config includes a new parameter we must set, `max_sequences_per_pack`, this informs the model of the maximum number of sequences it will need to 'unpack' in the model output. It also allows us to clearly define the `num_labels` and `problem_type` for this model.

The problem type is essential to define here, as switching between methods used by different types of classification requires it within the custom model.

In [ ]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(model_checkpoint)
config.max_sequences_per_pack = max_seq_per_pack
config.num_labels = num_labels
config.problem_type = problem_type

Now we can instantiate the model class with the config, loading the weights from the model checkpoint.

In [ ]:
import torch
import numpy as np
torch.manual_seed(43)
np.random.seed(43)

from models.modeling_bert_packed import PipelinedPackedBertForSequenceClassification

model = PipelinedPackedBertForSequenceClassification.from_pretrained(
    model_checkpoint, config=config).train()

print(config)

The warning is telling us we are throwing away some weights and randomly initializing others. This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

We can first test the model on CPU and observe that the output logits have now the size [batch_size x max_seq_per_pack, 2] = [12, 2] with this notebook default values.

In [ ]:
from transformers.data.data_collator import default_data_collator
import torch

model.float()
loader = torch.utils.data.DataLoader(packed_train_dataset,
                             batch_size=micro_batch_size,
                             shuffle=True,
                             drop_last=True,
                             collate_fn=default_data_collator)
data = next(iter(loader))
outputs = model(**data)
print(outputs)

Now let's prepare the model for IPU

First, we set the model in half precision:

In [ ]:
model.half()

For validation, we need to define a function to compute the metrics from the predictions, which will just use the `metric` we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits (our just squeeze the last axis in the case of STS-B). To ignore the `-100` labels from uncomplete packs, we use a boolean mask.

In [ ]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    
#   Remove the padding labels
    mask = (labels != -100)
    labels = labels[mask]
    
    if task != "stsb":
        predictions = np.argmax(predictions, axis=-1)
    else:
        predictions = predictions[:, 0]
    
    predictions = predictions[mask]
    
    return metric.compute(predictions=predictions, references=labels)

Next, we need to define the `IPUConfig`, which is a class that specifies attributes and configuration parameters to compile and put the model on the device. We initialize it with one config name or path, which we set earlier. Then we use it to set the mode attribute `model.ipu_config` 

In [ ]:
from optimum.graphcore import IPUConfig, IPUTrainer, IPUTrainingArguments

ipu_config = IPUConfig.from_pretrained(
    ipu_config_name,
    executable_cache_dir = executable_cache_dir,
    gradient_accumulation_steps=gradient_accumulation_steps,
    replication_factor=1,
    device_iterations = device_iterations,
    inference_device_iterations= 16,
    inference_replication_factor=1
)

The IPUTrainingArguments define any custom parameter modification we want to do, such as the initial learning rate for the model. It also allows other options, such as dataloader paramters, micro batch sizes and an automatic push to the Huggingface Hub (if credentials were set up earlier) to happen at given intervals.

These arguments are passed to the `IPUTrainer` which wraps the model training and evaluation process into a simple single-line process, doing all of the heavy lifting for us regarding training and evaluation loops, device assignment, optimiser definition, dataloading etc.

Note that only some arbitrary hyperparameter tuning was performed for this task. Other tasks and datasets may require further tuning to get the most optimal results.

In [ ]:
from transformers import default_data_collator

args = IPUTrainingArguments(
    "./"+f"{model_name}-{task}",
    num_train_epochs=2,
    per_device_train_batch_size=micro_batch_size,
    per_device_eval_batch_size=2,
    learning_rate=9e-5,
    warmup_ratio=0.1,
    weight_decay=0,
    lr_scheduler_type = "cosine",
    metric_for_best_model=metric_name,
    dataloader_drop_last=True,
    dataloader_mode="async_rebatched",
    logging_steps=1,
    pod_type=pod_type,
    gradient_accumulation_steps=gradient_accumulation_steps,
    push_to_hub=True
)


trainer = IPUTrainer(
    model,
    ipu_config,
    args,
    train_dataset=packed_train_dataset,
    eval_dataset=packed_val_dataset,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics
)

Then, to train the model we can simply call the `train()` method:

In [ ]:
trainer.train()

***About the performance:*** `IPUTrainer` doesn't take into account that we have packed data samples when computing the speed metrics. It treats a 'sample' as a single input to the model, i.e. one **pack**.

So the actual throughput estimation can be obtained by multiplying the `samples_per_second` by the average packing factor (the average number of samples per pack) of the dataset. These were obtained in the `packing_algorithm` section: `5.15` for `sst2` training set and `5.77` for validation set.

Next, we can evaluate the model by simply calling the `evaluate()` method:

In [ ]:
trainer.evaluate()

To see how your model fared you can compare it to the [GLUE Benchmark leaderboard](https://gluebenchmark.com/leaderboard).

You can now upload the result of the training to the Hub if you successfully logged in at the beginning of this notebook, just execute this instruction:

In [ ]:
trainer.push_to_hub()

You can also save the model locally:

In [ ]:
trainer.save_model("./"+f"{model_name}-{task}")

You have now successfully fine-tuned and evaluated your speed-optimised model for text classification using packing!

## Faster inference:

This section demonstrates how to perform faster, batched inference with a large number of samples using packing.

When training, the packing factor does affect the convergence the same way as a large increase in batch size would do. However, for inference, we are free to use a bigger packing factor to speed it up.
Let's try it on `sst2` with `max_seq_per_pack = 12`.

In [ ]:
max_seq_per_pack = 12

To have enough examples, we will reuse the training set.

In [ ]:
dataset = load_dataset("glue", "sst2")
encoded_dataset = dataset.map(preprocess_function, batched=True)
inference_dataset = encoded_dataset['train'] # Use the train set again, to have enough examples

inference_packed_dataset = PackedDatasetCreator(
    tokenized_dataset = encoded_dataset['train'],
    max_sequence_length = max_seq_length,
    max_sequences_per_pack = max_seq_per_pack,
    inference = True,
    problem_type = problem_type,
    custom_label_key='label').create()

We can see that the average packing factor `6.7` is not close to the maximum now (12), this is still an improvement compared to the previous `5.7`.

Let's also modify the configuration of the model for inference. For speed up, we can us a single IPU and 4 replicas by changing `layers_per_ipu` , `inference_replication_factor` and `ipus_per_replica` and also use a larger `batch-size`.

In [ ]:
ipu_config.layers_per_ipu = [12]
ipu_config.inference_device_iterations = 32
ipu_config.inference_replication_factor = 4
ipu_config.ipus_per_replica = 1

Let's load the checkpoint we saved earlier to run the inference on:

In [ ]:
model_checkpoint = f"./{model_name}-{task}"

# Load from Huggingface Hub instead:
# model_checkpoint = '<your_username>/{model_checkpoint}-{model_task}'

model = PipelinedPackedBertForSequenceClassification.from_pretrained(model_checkpoint, config=config)

In [ ]:
args = IPUTrainingArguments(
    "/tmp/"+f"{model_name}-{task}-fast-inf",
    per_device_eval_batch_size=8,
    dataloader_mode="async_rebatched",
    dataloader_drop_last=True,
    logging_steps=10,
    pod_type=pod_type
)

trainer = IPUTrainer(
    model,
    ipu_config,
    args,
    eval_dataset=inference_packed_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.evaluate()

As before, to get a correct throughput estimation we need to multiply `eval_samples_per_second` by the average packing factor (6.72). For example, if the inference throughput over a run is `4200 samples/s`, the actual throughput is `4200 * 6.72 = 28224 samples/s` 